In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import pickle
import json
import numpy as np

In [ ]:
print(tfds.__version__)

In [38]:
text_lines = []

for book in os.listdir('./books/'):
    with open(f'./books/{book}', encoding='iso 8859-1') as f:
        text_lines += f.readlines()

text_lines = [a.lower() for a in text_lines]

filtered_text_lines = []

for line in text_lines:
    if line == '\n':
        pass
        # print('linha vazia')
    else:
        spaces = line.count(' ')
        total = len(line)

        empty_space = spaces/total

        if empty_space >= 0.5:
            pass

        else:
            filtered_text_lines.append(line)

with open('./text.txt', 'w') as f:
    f.writelines(filtered_text_lines)


# www.nead.unama.br

In [ ]:
# tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(text_lines, target_vocab_size=2**13)

In [ ]:
# with open('tokenizer', 'wb') as f:
#     f.write(pickle.dumps(tokenizer))

In [39]:
with open('tokenizer', 'rb') as f:
    tokenizer = pickle.loads(f.read())

vocab_size = tokenizer.vocab_size

In [44]:
# text = ''.join(filtered_text_lines)

In [45]:
# tokens = tokenizer.encode(text)

In [46]:
# with open('tokens.json', 'w') as f:
    # f.write(json.dumps(tokens))

In [47]:
with open('tokens.json', 'r') as f:
    tokens = json.loads(f.read())

In [ ]:
# model = keras.Sequential([
#     layers.Embedding(input_dim=vocab_size, output_dim=256, input_shape=(100,)),
#     layers.LSTM(512),
#     layers.Dense(2048),
#     layers.Dense(vocab_size, activation='softmax')
# ])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# model.summary()

In [ ]:
# model.save('model.h5')

In [121]:
model = keras.models.load_model('model.h5')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 256)          2102784   
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1574912   
_________________________________________________________________
dense (Dense)                (None, 2048)              1050624   
_________________________________________________________________
dense_1 (Dense)              (None, 8214)              16830486  
Total params: 21,558,806
Trainable params: 21,558,806
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for i in range(0, len(tokens), 1024):
    x = []
    y = []

    for ii in range(i, i+1024):
        x += [tokens[ii:(ii+100)]]
        y.append(tokens[ii+100])

    
    x = np.array(x)
    y = keras.utils.to_categorical(y, num_classes=vocab_size)

    model.fit(x, y, batch_size=64)

In [118]:
from sty import fg, bg, ef, rs
from tqdm import tqdm

def complete_phrase(phrase, tokens=100, colors=True):
    print('-'*100)
    print('ENTRADA\n')
    print(phrase)

    string = phrase

    text = tokenizer.encode(phrase)


    for _ in tqdm(range(tokens)):
        prediction = model.predict(np.array([text]))[0]

        max_i = np.argmax(prediction)
        max_value = prediction[max_i]

        if colors:
            if max_value < 0.5:
                color = bg(int(255*abs(0.5-max_value)*2), 0, 0)
            else:
                color = bg(0, int(255*abs(0.5-max_value)*2), 0)

            string += color + tokenizer.decode([max_i])
        
        else:
            string += tokenizer.decode([max_i])

        text += [max_i]

    print('-'*100)
    print('SAIDA\n')
    # print(tokenizer.decode(text))
    print(string)

In [124]:
complete_phrase('maicon lia um livro quando', tokens=100, colors=False)

  2%|▏         | 2/100 [00:00<00:05, 17.02it/s]----------------------------------------------------------------------------------------------------
ENTRADA

maicon lia um livro quando
100%|██████████| 100/100 [00:10<00:00,  9.62it/s]----------------------------------------------------------------------------------------------------
SAIDA

maicon lia um livro quando, estando escuro, escuro, escuro,
circulava o aposento.
       o relógio de rosalina foi conduzido pela porta do quarto de rosalina,
que estava fechada dentro de uma alcova, e o outro, que se achava diante
dela, parecia que o corredor estava escuro.
       o pataca acendeu o gás, e foi sentar-se à janela, com um ar de
tristeza, que parecia estar muito disposta a receber o obséquio de adelaide.
       o bacharel

